# Соревнование: San Francisco Crime Classification
*Given time and location, you must predict the category of crime that occurred. Kaggle is hosting this competition for the machine learning community to use for fun and practice.*

https://www.kaggle.com/c/sf-crime

Этот ноутбук структурирован таким образом, чтобы облегчить совместную работу над соревнованием. За некоторыми исключениями, каждый участник работает только в разделе 4. При этом ни одна из строк кода любого из разделов не является постоянной, если в ней нет необходимости, ее стоит исключить, если один из участников написал полезный код, который может пригодится остальным, то этот когда имеет смысл перенести из персонального раздела в общие. Ноутбук имеет следующую структуру:
1. Подготовка работы. Загрузка библиотек и настройка отображения
+ Загрузка и очистка данных
+ Общие функции
+ Персональный раздел
+ Модель на тестовой выборке 

### 1. Подготовка работы. Загрузка библиотек и настройка отображения
Импорты и настроийки, которые необходимы для шаблона.

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
import time
import datetime
import csv

%matplotlib inline

### 2. Загрузка, очистка данных. Общие константы
Обучающая и тренировочная выборки надоходятся в папке `data`. 

In [53]:
train = pd.read_csv('./data/train.csv')

#### 2.1. Очистка данных
В то время, как в `train` находятся сырые данные, `X_train` содержит те же колонки, что и `test`, поэтому многие преобразования над обеими переменными могут выполняться одной функцией.

In [54]:
# необходимо принять волевое решение, что делать с данными у которых левые координаты?
# пока их исключаем
train = train[train.Y < 40]
# эти колонки содержаться в тестовой выбоке
test_features = pd.read_csv('./data/test.csv', nrows=1, index_col='Id').columns.tolist()
# теперь X_train с правильными колонками
X_train = train[test_features]
# целевая переменная, не путать с координатой
y = train.Category
# список происшествий, колонки для файла с результатами 
out_features = pd.read_csv('./data/sampleSubmission.csv', nrows=1, index_col='Id').columns.tolist()
# количество классов происшествий
classes = len(out_features)

#### 2.2. Общие константы
Константы, которые используются для обработки данных.

In [55]:
# данные
train_rows = X_train.shape[0]
train_features = train.columns.tolist()
test_features = pd.read_csv('./data/test.csv', nrows=1, index_col='Id').columns.tolist()

# другие
random_state = 19

### 3. Общие функции
Это функции для операций с данными.
Сюда надо написать функции для записи моделей и их параметров в файл (хранить в json?), и пр.

In [56]:
# добавление колонок к выборке
# хорошо бы сделать, чтоб работало для разных типов
def add_features_to_X(X, X_add):
    assert X.shape[0] == X_add.shape[0]
    return pd.concat([X, X_add], axis=1, join_axes=[X.index])

# расп-сить колонку y.Category в матрицу с колонками out_features
def convert_y_to_matrix(Category):
    pass

# запись результатов в файл
# имена колонок содержатся в out_features
def write_results(filename, results):
    pass    #


### 4. Персональный раздел. 
Место для творчества. В этом пункте можно творить что угодно! 

Возможно, стоит указывать свой `id` для наиболее полезных функций и т.п.

В конце раздела передаем свой классификатор в переменную `clf`

#### 4.1. Мои импорты

In [57]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

#### 4.2. Мои константы

In [58]:
# то, чем пользуюcь только я
random_state = 42
n_jobs = 4
learning_rate = 0.1

#### 4.3. Мои функции

In [59]:
def encode_data(train, test):
    le = LabelEncoder()
    
    train['DayOfWeek'] = le.fit_transform(train.DayOfWeek)
    test['DayOfWeek'] = le.transform(test.DayOfWeek)
    
    train['PdDistrict'] = le.fit_transform(train.PdDistrict)
    test['PdDistrict'] = le.transform(test.PdDistrict)
    
    #Заменяем адреса отсутствующие в тестовой выборке на самое частое значение в обучающей: (исправить и заменить на ближайший адрес по координатам?)
    test.loc[~test.Address.isin(np.unique(train.Address)),['Address']] = '800 Block of BRYANT ST'
    train['Address'] = le.fit_transform(train.Address)
    test['Address'] = le.transform(test.Address)

    return train, test

def get_dates(X):
    X['Dates'] = pd.to_datetime(X.Dates, yearfirst=True)
    X['Year'] = X.Dates.dt.year
    X['Month'] = X.Dates.dt.month
    X['Hour'] = X.Dates.dt.hour
    return X


#### 4.4. Моя модель
Моя крутая модель, ее запредельные метрики и наглядные графики.

In [60]:
#Загрузим и сразу преобразуем тестовую выборку для правильного кодирования данных:
test = pd.read_csv('./data/test.csv')

X, X_test = encode_data(X_train.copy(), test.copy())
X, X_test = get_dates(X.copy()), get_dates(X_test.copy())
# настройка кросс-валидации
cv = KFold(n=train_rows, n_folds=5, shuffle=True, random_state=random_state)

# мой классификатор
clf = GradientBoostingClassifier(n_estimators=20, random_state=random_state)

In [44]:
# обучение моего классификаторв и проверка качества
features = ['DayOfWeek', 'PdDistrict','X','Y','Address', 'Year', 'Month', 'Hour']

start_time = datetime.datetime.now()
logloss = cross_val_score(clf, X[features], y, cv=cv, scoring='log_loss').mean()
duration = (datetime.datetime.now() - start_time).total_seconds()

print('Time elapsed:', duration)
print('average score (5 folds): %s' % logloss)

Time elapsed: 5253.39222
average score (5 folds): -2.52753946897


In [61]:
clf.fit(X[features], y)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)

### 5. Тестирование
О том, что надо скормить своему классификатору тестовую выборку.

#### 5.1. Загрузка и константы тестовой выборки

#### 5.2. Обработка тестовой выборки

#### 5.3. Предсказания классификатора
Пропускаем тестовую выборку через свой классификатор и записываем результаты в файл. 

In [62]:
pred = clf.predict_proba(X_test[features])

In [63]:
pred = np.round(pred, decimals=7)
submit = pd.DataFrame(pred, columns=clf.classes_)
submit['Id'] = test.Id

In [64]:
submit.to_csv('gb20new.csv',index=False)